# Project Tasks

### Test 1: Play each sound and record human matching

In [61]:
import numpy as np
from glob import glob
from pathlib import Path
from scipy.io import wavfile
from IPython.display import Audio

data_folder = Path().resolve() / "Audio Files"

twelve_test_files = glob(f'{data_folder}\Twelve Test\*.wav')
twelve_train_files = glob(f'{data_folder}\Twelve Train\*.wav')
zero_test_files = glob(f'{data_folder}\Zero Test\*.wav')
zero_train_files = glob(f'{data_folder}\Zero Train\*.wav')

#### "Zero" Training 

In [57]:
zero_train_arrays = []
for file_path in zero_train_files:
    sample_rate, audio_data = wavfile.read(file_path)
    zero_train_arrays.append(np.array(audio_data))
print(f"Sample rate: {sample_rate}")
print(f"Number of audio files: {len(zero_train_arrays)}")

Sample rate: 48000
Number of audio files: 18


In [58]:
recording_num = 10
Audio(zero_train_arrays[recording_num], rate=sample_rate)

75776


#### "Zero" Testing 

In [59]:
zero_test_arrays = []
for file_path in zero_test_files:
    sample_rate, audio_data = wavfile.read(file_path)
    zero_test_arrays.append(np.array(audio_data))
print(f"Sample rate: {sample_rate}")
print(f"Number of audio files: {len(zero_test_arrays)}")

Sample rate: 48000
Number of audio files: 18


In [60]:
recording_num = 10
Audio(zero_test_arrays[recording_num], rate=sample_rate)

91136


#### "Twelve" Training 

In [64]:
twelve_train_arrays = []
for file_path in twelve_train_files:
    sample_rate, audio_data = wavfile.read(file_path)
    twelve_train_arrays.append(np.array(audio_data))
print(f"Sample rate: {sample_rate}")
print(f"Number of audio files: {len(twelve_train_arrays)}")

Sample rate: 48000
Number of audio files: 18


In [65]:
recording_num = 10
Audio(twelve_train_arrays[recording_num], rate=sample_rate)

#### "Twelve" Testing 

In [66]:
twelve_test_arrays = []
for file_path in twelve_test_files:
    sample_rate, audio_data = wavfile.read(file_path)
    twelve_test_arrays.append(np.array(audio_data))
print(f"Sample rate: {sample_rate}")
print(f"Number of audio files: {len(twelve_test_arrays)}")

Sample rate: 48000
Number of audio files: 18


In [67]:
recording_num = 10
Audio(twelve_test_arrays[recording_num], rate=sample_rate)

### Test 2: Play a sound, plot, and answer how long is 256 samples in time.